In [ ]:
import requests
from urllib import parse
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def getHoliday(year):
    url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo"
    api_key_utf8 = 'yNn1SVQGgJeAnb5M8oyK//N9wu0L7h/Ou+1omKfWOhCaZVWeFBvyEdJ0ATHPIp4FYKrIF7jD3CeZF+shPVCEXQ=='
    api_key_decode = parse.unquote(api_key_utf8, encoding='utf-8')

    item_list = []
    for month in range(1, 13):
        params = {
            'ServiceKey': api_key_decode,
            'solYear': year,
            'solMonth': f'{month:02d}',
            'numOfRows': 100
        }

        response = requests.get(url, params=params)
        xml = BeautifulSoup(response.text, 'lxml')
        items = xml.find_all('item')

        for item in items:
            try:
                name = item.find('datename').text.strip()
                date = datetime.strptime(item.find('locdate').text.strip(), '%Y%m%d')
                item_list.append({'날짜': date, '이름': name})
            except:
                continue

    return pd.DataFrame(item_list)



# 2017-2025 합치기
df = pd.DataFrame(columns=['날짜', '이름'])
for year in range(2017, 2026):
    df = pd.concat([df, getHoliday(year)], ignore_index=True)

# 중복 제거 및 정렬
df = df.drop_duplicates().sort_values(by='날짜').reset_index(drop=True)

df

C:\Users\user\AppData\Local\Temp\ipykernel_13796\2148749692.py:22: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  xml = BeautifulSoup(response.text, 'lxml')
C:\Users\user\AppData\Local\Temp\ipykernel_13796\2148749692.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

,날짜,이름
0,2017-01-01,신정
1,2017-01-27,설날
2,2017-01-28,설날
3,2017-01-29,설날
4,2017-01-30,대체공휴일
...,...,...
160,2025-10-06,추석
161,2025-10-07,추석
162,2025-10-08,대체공휴일
163,2025-10-09,한글날


In [16]:
df

,날짜,이름
0,2017-01-01,신정
1,2017-01-27,설날
2,2017-01-28,설날
3,2017-01-29,설날
4,2017-01-30,대체공휴일
...,...,...
160,2025-10-06,추석
161,2025-10-07,추석
162,2025-10-08,대체공휴일
163,2025-10-09,한글날


In [14]:
df.to_csv('../results/공휴일정보_2017_2025.csv', index=False, encoding='utf-8-sig')

In [17]:
getHoliday(2024) ### 제대로 있는지 확인용

C:\Users\user\AppData\Local\Temp\ipykernel_13796\2148749692.py:22: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  xml = BeautifulSoup(response.text, 'lxml')


,날짜,이름
0,2024-01-01,1월1일
1,2024-02-09,설날
2,2024-02-10,설날
3,2024-02-11,설날
4,2024-02-12,대체공휴일(설날)
5,2024-03-01,삼일절
6,2024-04-10,국회의원선거
7,2024-05-05,어린이날
8,2024-05-06,대체공휴일(어린이날)
9,2024-05-15,부처님오신날


In [ ]:
### 우리가 공휴일 정보만 쓰면 되는건지 확인용 코드 2024년도만 확인 ###
import requests
from urllib import parse
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def get_special_days(year):
    api_key_utf8 = 'yNn1SVQGgJeAnb5M8oyK//N9wu0L7h/Ou+1omKfWOhCaZVWeFBvyEdJ0ATHPIp4FYKrIF7jD3CeZF+shPVCEXQ=='
    api_key_decode = parse.unquote(api_key_utf8)

    base_url = "http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/"
    endpoints = {
        # '기념일': 'getAnniversaryInfo', ### 보건의 날, 과학의 날 등등 필요없음
        '공휴일': 'getRestDeInfo',        ### 우리가 필요로 하는 정보
        '국경일': 'getHoliDeInfo',        ### 국경일 및 공휴일 정보 
        # '24절기': 'get24DivisionsInfo', ### 소한, 대한, 입춘,, 대설 등 필요없음
        # '잡절': 'getSundryDayInfo'      ### 정월대보름, 한식, 단오, 초복 등등 필요X
    }

    all_items = []

    for label, endpoint in endpoints.items():
        for month in range(1, 13):
            params = {
                'ServiceKey': api_key_decode,
                'solYear': year,
                'solMonth': f'{month:02d}',
                'numOfRows': 100
            }
            response = requests.get(base_url + endpoint, params=params)
            soup = BeautifulSoup(response.text, 'lxml')
            items = soup.find_all('item')

            for item in items:
                try:
                    date_str = item.find('locdate').text.strip()
                    name = item.find('datename').text.strip()
                    locdate = datetime.strptime(date_str, '%Y%m%d')
                    all_items.append({'날짜': locdate, '이름': name, '특일구분': label})
                except:
                    continue  # 필드 누락 등 예외 처리

    df = pd.DataFrame(all_items).sort_values('날짜').reset_index(drop=True)
    return df

df_special = get_special_days(2024)
df_special

C:\Users\user\AppData\Local\Temp\ipykernel_13796\796083300.py:32: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.text, 'lxml')


,날짜,이름,특일구분
0,2024-01-01,1월1일,공휴일
1,2024-01-01,1월1일,국경일
2,2024-02-09,설날,국경일
3,2024-02-09,설날,공휴일
4,2024-02-10,설날,국경일
5,2024-02-10,설날,공휴일
6,2024-02-11,설날,국경일
7,2024-02-11,설날,공휴일
8,2024-02-12,대체공휴일(설날),국경일
9,2024-02-12,대체공휴일(설날),공휴일


In [ ]:
# df_special.to_csv('특일정보.csv', encoding='utf-8-sig', index=False)